In [8]:
import csv
import pandas as pd
import numpy as np
from tpot import TPOTClassifier

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import joblib

file_feature = "./csv/endometrium.csv"
file_train = "./csv/train.csv"
file_validate = "./csv/validation.csv"
file_test = "./csv/test.csv"

f = open(file_feature)
csv_f = csv.reader(f)
features = next(csv_f)
dataset = pd.read_csv(file_feature, names=features, usecols=range(1,6098), dtype=np.float64, skiprows=1, low_memory=False)
f = open(file_train)
csv_f = csv.reader(f)
features = next(csv_f)
dataset_train = pd.read_csv(file_train, names=features, usecols=range(1,1), dtype=np.float64, skiprows=1, low_memory=False)

with open('./csv/train.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    train_list = [row[1] for row in reader]
with open('./csv/validation.csv','r') as csvfile:
    reader = csv.DictReader(csvfile)
    validation_list = [row['patient'] for row in reader]
with open('./csv/test.csv','r') as csvfile:
    reader = csv.DictReader(csvfile)
    test_list = [row['patient'] for row in reader]

dataset['outcome'] = pd.to_numeric(dataset['outcome'],errors='coerce')
array_OG = dataset.values
print(array_OG.shape)
train_list = train_list[1:]
validation_list = validation_list[0:]
test_list = test_list[0:]
#print(test_list)
#print(train_list)
#print(validation_list)

def cat_str(num_list):
    n_list = []
    for i in num_list:
        temp = i[12:]
        n_list.append(temp)
    n_list = [int(x) for x in n_list]
    return n_list

train_list = cat_str(train_list)
validation_list = cat_str(validation_list)
test_list = cat_str(test_list)

#print(train_list)
#print(validation_list)
#print(test_list)
#print(len(test_list))

train_feature = []
validate_feature = []
test_feature = []
count = 1
for i in range(len(array_OG)):
    num = i + 1
    if num in train_list:
        train_feature.append(array_OG[i])
    elif num in validation_list:
        validate_feature.append(array_OG[i])
    elif num in test_list:
        #print(count)
        count = count + 1
        test_feature.append(array_OG[i])
        #print(num)
        #print(array_OG[i,6096])
        
train_feature = np.array(train_feature)
validate_feature = np.array(validate_feature)
test_feature = np.array(test_feature)

train_feature = pd.DataFrame(train_feature)
train_feature.dropna(axis=1, thresh=2, inplace=True)
train_feature.dropna(how='all',thresh = 20,inplace=True)
train_feature = np.array(train_feature)
wh_inf = np.isinf(train_feature)
train_feature[wh_inf]=0
wh_nan = np.isnan(train_feature)
train_feature[wh_nan]=0

validate_feature = pd.DataFrame(validate_feature)
validate_feature.dropna(axis=1, thresh=2, inplace=True)
#validate_feature.dropna(how='all',thresh = 20,inplace=True)
validate_feature = np.array(validate_feature)
wh_inf = np.isinf(validate_feature)
validate_feature[wh_inf]=0
wh_nan = np.isnan(validate_feature)
validate_feature[wh_nan]=0

test_feature = pd.DataFrame(test_feature)
test_feature.dropna(axis=1, thresh=2, inplace=True)
#test_feature.dropna(how='all',thresh = 20,inplace=True)
test_feature = np.array(test_feature)
wh_inf = np.isinf(test_feature)
test_feature[wh_inf]=0
wh_nan = np.isnan(test_feature)
test_feature[wh_nan]=0

#only use image features
X_train = train_feature[:,:6093]
Y_train = train_feature[:,6093]
Y_train = Y_train.astype('int32')

X_validate = validate_feature[:,:6093]
Y_validate = validate_feature[:,6093]
Y_validate = Y_validate.astype('int32')

X_test = test_feature[:,:6093]
Y_test = test_feature[:,6093]
Y_test = Y_test.astype('int32')
seed = 7

np.random.seed(seed)
np.random.shuffle(X_train) 
np.random.seed(seed)
np.random.shuffle(Y_train)

print(Y_test)

(849, 6097)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1]


In [9]:
import csv
import pandas as pd
import numpy as np
from sklearn import model_selection
from tpot import TPOTClassifier
import tools

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier
#from pyearth import Earth
from sklearn.cross_decomposition import PLSRegression


from sklearn.feature_selection import SelectKBest

from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from skfeature.function.statistical_based import t_score
from skfeature.function.statistical_based import gini_index
from skfeature.function.similarity_based import fisher_score
from skfeature.function.similarity_based import reliefF

from sklearn.feature_selection import mutual_info_classif
from skfeature.function.information_theoretical_based import LCSI
from skfeature.function.information_theoretical_based import MIM
from skfeature.function.information_theoretical_based import MIFS
from skfeature.function.information_theoretical_based import MRMR
from skfeature.function.information_theoretical_based import CIFE
from skfeature.function.information_theoretical_based import JMI
from skfeature.function.information_theoretical_based import CMIM
from skfeature.function.information_theoretical_based import ICAP
from skfeature.function.information_theoretical_based import DISR

from scipy.stats import wilcoxon

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample

from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
# CLASSIFICATION METHODS
models = []
models.append(('GLM', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('BY', GaussianNB()))
models.append(('SVM', SVC(probability=True)))
models.append(('BAG', BaggingClassifier()))
models.append(('NNet', MLPClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('BST', AdaBoostClassifier()))
models.append(('KNN', KNeighborsClassifier()))
seed = 7

In [10]:
#crossvalidation on trainset and select the best model on validation set, test on test set
import xlwt

num_fea = 10
sel = []
sel.append(('CHSQ', SelectKBest(chi2, k=num_fea)))
sel.append(('ANOVA', SelectKBest(f_classif, k=num_fea)))
sel.append(('TSCR', SelectKBest(t_score.t_score, k=num_fea)))
sel.append(('FSCR', SelectKBest(fisher_score.fisher_score, k=num_fea)))
sel.append(('RELF', SelectKBest(reliefF.reliefF, k=num_fea)))

output = open("endohand10.txt","w")
file = 'fea10.xls'


book = xlwt.Workbook()
sheet = book.add_sheet('train_avg_auc')
sheet_train = book.add_sheet('train_auc')
sheet_validate = book.add_sheet('validate_auc')
sheet_test = book.add_sheet('test_auc')


from sklearn.externals import joblib
#from imblearn.over_sampling import SMOTE
# 定义SMOTE模型，random_state相当于随机数种子的作用
kfold = model_selection.KFold(n_splits=5, random_state=seed)
scaler = MinMaxScaler()
sc = scaler.fit(X_train)
X_train = sc.transform(X_train)
X_validate = sc.transform(X_validate)
X_test = sc.transform(X_test)
r = 0
c = 0
for name, model in models:
    for kind, selection in sel:
        #make_pipeline(MinMaxScaler(), selection, model)
        
        se = selection.fit(X_train,Y_train)
        X_train_ = se.transform(X_train)
        X_validate_ = se.transform(X_validate)
        X_test_ = se.transform(X_test)
        
        cv_results = model_selection.cross_val_score(model, X_train_, Y_train, scoring='roc_auc', cv=kfold)
        
        sheet.write(r,c,cv_results.mean())
        print(r,c)

        print("###########################################")
        msg = "%s %s %s: %f (%f)\n" % ("TRAIN_AUC", kind, name, cv_results.mean(), cv_results.std())
        print(msg)
        output.write(msg)
        model.fit(X_train_,Y_train)
        joblib.dump(model,'./handpkl/Endo'+name+kind+'10.pkl')
        
        Y_pred = model.predict(X_train_)
        #prob = model.decision_function(X_train_)
        print("Accuracy: " + repr(accuracy_score(Y_train, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_train, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_train, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_train, Y_pred)))
        print("AUC: " + repr(roc_auc_score(Y_train, Y_pred)))
        print("Sensitivity: " + repr(recall_score(Y_train, Y_pred)))
        tn, fp, fn, tp = confusion_matrix(Y_train, Y_pred).ravel()
        print("Specificity: " + repr(tn / (tn + fp)))
        
        sheet_train.write(r,c,roc_auc_score(Y_train, Y_pred))

        Y_pred = model.predict(X_validate_)
        #prob = model.decision_function(X_validate_)
        print("Accuracy: " + repr(accuracy_score(Y_validate, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_validate, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_validate, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_validate, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_validate,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_validate,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_validate,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_validate.write(r,c,roc_auc_score(Y_validate,Y_pred))
        
        Y_pred = model.predict(X_test_)
        #prob = model.decision_function(X_test_)
        print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
        print("AUC: " + repr(roc_auc_score(Y_test, Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_test,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
        print(tn,fp,fn,tp)
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_test.write(r,c,roc_auc_score(Y_test,Y_pred))
        
        r = r + 1
    c = c + 1
    r = 0

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

0 0
###########################################
TRAIN_AUC CHSQ GLM: 0.804854 (0.030327)

Accuracy: 0.7303030303030303
Average Precision Score: 0.6235536149187464
Kappa: 0.4561916753073618
Hamming Loss: 0.2696969696969697
AUC: 0.7286716661716661
Sensitivity: 0.7128378378378378
Specificity: 0.7445054945054945
Accuracy: 0.7710843373493976
Average Precision Score: 0.6806504212338074
Kappa: 0.5360400117681672
Hamming Loss: 0.2289156626506024
AUC0.7663742690058479
Sensitivity0.7105263157894737
Specificity0.8222222222222222
Accuracy: 0.6746987951807228
Average Precision Score: 0.5786745513264809
Kappa: 0.3513748191027497
Hamming Loss: 0.3253012048192771
AUC: 0.6774853801169589
Sensitivity0.7105263157894737
29 16 11 27
Specificity0.6444444444444445
1 0
###########################################
TRAIN_AUC ANOVA GLM: 0.802503 (0.019931)

Accuracy: 0.7378787878787879
Average Precision Score: 0.6324358331278747
Kappa: 0.46895987200714373
Hamming Loss: 0.26212121212121214
AUC: 0.7339619839619841
S

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

2 0
###########################################
TRAIN_AUC TSCR GLM: 0.803697 (0.021658)

Accuracy: 0.7393939393939394
Average Precision Score: 0.6335153411495693
Kappa: 0.47352799317330163
Hamming Loss: 0.2606060606060606
AUC: 0.7369134244134243
Sensitivity: 0.7128378378378378
Specificity: 0.760989010989011
Accuracy: 0.7831325301204819
Average Precision Score: 0.6884908053265694
Kappa: 0.564938846825859
Hamming Loss: 0.21686746987951808
AUC0.7836257309941521
Sensitivity0.7894736842105263
Specificity0.7777777777777778
Accuracy: 0.7590361445783133
Average Precision Score: 0.6590981149478297
Kappa: 0.5205083766608898
Hamming Loss: 0.24096385542168675
AUC: 0.7634502923976608
Sensitivity0.8157894736842105
32 13 7 31
Specificity0.7111111111111111


/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

...

1 7
###########################################
TRAIN_AUC ANOVA RF: 0.718903 (0.029716)

Accuracy: 0.9787878787878788
Average Precision Score: 0.9724115680981879
Kappa: 0.9569575911157486
Hamming Loss: 0.021212121212121213
AUC: 0.9766669141669142
Sensitivity: 0.956081081081081
Specificity: 0.9972527472527473
Accuracy: 0.7108433734939759
Average Precision Score: 0.6173324878461213
Kappa: 0.4078478002378121
Hamming Loss: 0.2891566265060241
AUC0.7005847953216374
Sensitivity0.5789473684210527
Specificity0.8222222222222222
Accuracy: 0.6987951807228916
Average Precision Score: 0.6011499768633567
Kappa: 0.39203047172575445
Hamming Loss: 0.30120481927710846
AUC: 0.6956140350877195
Sensitivity0.6578947368421053
33 12 13 25
Specificity0.7333333333333333
2 7
###########################################
TRAIN_AUC TSCR RF: 0.724063 (0.021123)

Accuracy: 0.9787878787878788
Average Precision Score: 0.9724115680981879
Kappa: 0.9569575911157486
Hamming Loss: 0.021212121212121213
AUC: 0.9766669141669142


/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)


1 8
###########################################
TRAIN_AUC ANOVA BST: 0.725017 (0.021961)

Accuracy: 0.7772727272727272
Average Precision Score: 0.6744389975220646
Kappa: 0.5521685345543841
Hamming Loss: 0.22272727272727272
AUC: 0.7775653400653401
Sensitivity: 0.7804054054054054
Specificity: 0.7747252747252747
Accuracy: 0.7951807228915663
Average Precision Score: 0.6989721431626137
Kappa: 0.5916063675832127
Hamming Loss: 0.20481927710843373
AUC0.7988304093567251
Sensitivity0.8421052631578947
Specificity0.7555555555555555
Accuracy: 0.7108433734939759
Average Precision Score: 0.6121353836398225
Kappa: 0.4199184624344787
Hamming Loss: 0.2891566265060241
AUC: 0.7108187134502923
Sensitivity0.7105263157894737
32 13 11 27
Specificity0.7111111111111111
2 8
###########################################
TRAIN_AUC TSCR BST: 0.739537 (0.026226)

Accuracy: 0.7772727272727272
Average Precision Score: 0.6767878256786106
Kappa: 0.549339477155757
Hamming Loss: 0.22272727272727272
AUC: 0.774409711909712
Se

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)


2 9
###########################################
TRAIN_AUC TSCR KNN: 0.738552 (0.032250)

Accuracy: 0.7848484848484848
Average Precision Score: 0.6837889025389026
Kappa: 0.5667209113100082
Hamming Loss: 0.21515151515151515
AUC: 0.7844334719334718
Sensitivity: 0.7804054054054054
Specificity: 0.7884615384615384
Accuracy: 0.7590361445783133
Average Precision Score: 0.665424857324033
Kappa: 0.5126247798003524
Hamming Loss: 0.24096385542168675
AUC0.7552631578947369
Sensitivity0.7105263157894737
Specificity0.8
Accuracy: 0.6987951807228916
Average Precision Score: 0.6002466004368351
Kappa: 0.4018449120784088
Hamming Loss: 0.30120481927710846
AUC: 0.7038011695906432
Sensitivity0.7631578947368421
29 16 9 29
Specificity0.6444444444444445
3 9
###########################################
TRAIN_AUC FSCR KNN: 0.742798 (0.026827)

Accuracy: 0.7909090909090909
Average Precision Score: 0.6914902716711927
Kappa: 0.5783957932158199
Hamming Loss: 0.20909090909090908
AUC: 0.7899279774279774
Sensitivity: 0.78

In [11]:
#scaler = MinMaxScaler()
#X_train = scaler.fit_transform(X_train)
#X_validate = scaler.fit_transform(X_validate)
#X_test = scaler.fit_transform(X_test)

# WILCOXON SCORE FUNCTION
def takeSecond(elem):
    return elem[1]
def WLCX(data, target, n_selected_features):
    pval = []
    for num in range(len(data[1])):
        x = data[:,num]
        pval.append([num, wilcoxon(x,target)[1]])
    pval.sort(key=takeSecond)
    idx = []
    for i in range(n_selected_features):
        idx.append(pval[i][0])
    return idx

# MULTIVARIATE FEATURE SELECTION X CLASSIFICATION (10 fold CV)

# print('BEFORE')
MV_sel = []
MV_sel.append(('WLCX', WLCX(X_train, Y_train, n_selected_features=num_fea)))
print('WLCX')

for name, model in models:
    for kind, idx in MV_sel:
        # X_sel = X[:, idx[0:num_fea]]
        X_test_ = X_test[:,idx[0:num_fea]]
        X_validate_ = X_validate[:,idx[0:num_fea]]
        X_train_ = X_train[:, idx[0:num_fea]]
        # X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X_sel, Y, test_size=validation_size, random_state=seed)
        #kfold = model_selection.KFold(n_splits=10, random_state=seed)
        cv_results = model_selection.cross_val_score(model, X_train_, Y_train, cv=kfold, scoring='roc_auc')
        
        sheet.write(r,c,cv_results.mean())
        
        msg = "%s %s: %f (%f)\n" % (kind, name, cv_results.mean(), cv_results.std())
        print(msg)
        output.write(msg)
        model.fit(X_train_, Y_train)
        joblib.dump(model,'./handpkl/Endo'+name+kind+'10.pkl')
        
        Y_pred = model.predict(X_train_)
        #prob = model.predict_prob(X_train_)
        print("Accuracy: " + repr(accuracy_score(Y_train, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_train, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_train, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_train, Y_pred)))
        print("AUC: " + repr(roc_auc_score(Y_train, Y_pred)))
        print("Sensitivity: " + repr(recall_score(Y_train, Y_pred)))
        tn, fp, fn, tp = confusion_matrix(Y_train, Y_pred).ravel()
        print("Specificity: " + repr(tn / (tn + fp)))
        
        sheet_train.write(r,c,roc_auc_score(Y_train, Y_pred))
        
        Y_pred = model.predict(X_validate_)    
        #prob = model.predict_prob(X_validate_)
        print("Accuracy: " + repr(accuracy_score(Y_validate, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_validate, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_validate, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_validate, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_validate,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_validate,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_validate,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_validate.write(r,c,roc_auc_score(Y_validate,Y_pred))
        
        Y_pred = model.predict(X_test_)
        #prob = model.predict_prob(X_test_)
        print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_test,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_test,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_test.write(r,c,roc_auc_score(Y_test,Y_pred))
        
        r = r + 1
    c = c + 1
    r = 0

WLCX
WLCX GLM: 0.674343 (0.030123)

Accuracy: 0.6242424242424243
Average Precision Score: 0.5201921419026682
Kappa: 0.19413919413919412
Hamming Loss: 0.37575757575757573
AUC: 0.5914946539946541
Sensitivity: 0.27364864864864863
Specificity: 0.9093406593406593
Accuracy: 0.5903614457831325
Average Precision Score: 0.5039632213062777
Kappa: 0.12087227414330204
Hamming Loss: 0.40963855421686746
AUC0.5567251461988305
Sensitivity0.15789473684210525
Specificity0.9555555555555556
Accuracy: 0.6506024096385542
Average Precision Score: 0.5702404760743378
Kappa: 0.25824345146379046
Hamming Loss: 0.3493975903614458
AUC0.6225146198830409
Sensitivity0.2894736842105263
Specificity0.9555555555555556
WLCX LDA: 0.662152 (0.031260)

Accuracy: 0.6515151515151515
Average Precision Score: 0.5486397464658335
Kappa: 0.2585863322002109
Hamming Loss: 0.3484848484848485
AUC: 0.6228467478467479
Sensitivity: 0.34459459459459457
Specificity: 0.9010989010989011
Accuracy: 0.6506024096385542
Average Precision Score: 0.5

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

WLCX SVM: 0.674834 (0.031484)

Accuracy: 0.5893939393939394
Average Precision Score: 0.4853311103311103
Kappa: 0.10835709585435382
Hamming Loss: 0.4106060606060606
AUC: 0.550434362934363
Sensitivity: 0.17229729729729729
Specificity: 0.9285714285714286
Accuracy: 0.5662650602409639
Average Precision Score: 0.4798139928133587
Kappa: 0.06508135168961204
Hamming Loss: 0.43373493975903615
AUC0.5304093567251462
Sensitivity0.10526315789473684
Specificity0.9555555555555556
Accuracy: 0.6144578313253012
Average Precision Score: 0.5346781864299303
Kappa: 0.17258566978193135
Hamming Loss: 0.3855421686746988
AUC0.5809941520467835
Sensitivity0.18421052631578946
Specificity0.9777777777777777
WLCX BAG: 0.647322 (0.035188)

Accuracy: 0.7651515151515151
Average Precision Score: 0.7041834127926844
Kappa: 0.5024996595794347
Hamming Loss: 0.23484848484848486
AUC: 0.7397534897534898
Sensitivity: 0.49324324324324326
Specificity: 0.9862637362637363
Accuracy: 0.6867469879518072
Average Precision Score: 0.601902

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the op

WLCX NNet: 0.677250 (0.031541)



/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.6530303030303031
Average Precision Score: 0.5507348257348257
Kappa: 0.26107362863009687
Hamming Loss: 0.346969696969697
AUC: 0.6239048114048115
Sensitivity: 0.34121621621621623
Specificity: 0.9065934065934066
Accuracy: 0.6506024096385542
Average Precision Score: 0.5658845909955612
Kappa: 0.26142988646824183
Hamming Loss: 0.3493975903614458
AUC0.624561403508772
Sensitivity0.3157894736842105
Specificity0.9333333333333333
Accuracy: 0.6265060240963856
Average Precision Score: 0.536163228766459
Kappa: 0.2138710663000305
Hamming Loss: 0.37349397590361444
AUC0.6023391812865497
Sensitivity0.3157894736842105
Specificity0.8888888888888888
WLCX RF: 0.638436 (0.029834)

Accuracy: 0.7606060606060606
Average Precision Score: 0.6973300573300574
Kappa: 0.4927028604786923
Hamming Loss: 0.23939393939393938
AUC: 0.735001485001485
Sensitivity: 0.4864864864864865
Specificity: 0.9835164835164835
Accuracy: 0.6746987951807228
Average Precision Score: 0.5887427827654107
Kappa: 0.3182233039245512
Ha

In [12]:
MV_sel = []
MV_sel.append(('MIM', MIM.mim(X_train, Y_train, n_selected_features=num_fea)))
print('MIM')
MV_sel.append(('MIFS', MIFS.mifs(X_train, Y_train, n_selected_features=num_fea)))
print('MIFS')
MV_sel.append(('MRMR', MRMR.mrmr(X_train, Y_train, n_selected_features=num_fea)))
print('MRMR')
MV_sel.append(('CIFE', CIFE.cife(X_train, Y_train, n_selected_features=num_fea)))
print('CIFE')
MV_sel.append(('JMI', JMI.jmi(X_train, Y_train, n_selected_features=num_fea)))
print('JMI')
MV_sel.append(('CMIM', CMIM.cmim(X_train, Y_train, n_selected_features=num_fea)))
print('CMIM')
MV_sel.append(('ICAP', ICAP.icap(X_train, Y_train, n_selected_features=num_fea)))
print('ICAP')
MV_sel.append(('DISR', DISR.disr(X_train, Y_train, n_selected_features=num_fea)))

MIM
MIFS
MRMR
CIFE
JMI
CMIM
ICAP


In [13]:
for name, model in models:
    for kind, idx in MV_sel:

        X_test_ = X_test[:,idx[0:num_fea][0]]
        X_validate_ = X_validate[:,idx[0:num_fea][0]]
        X_train_ = X_train[:,idx[0:num_fea][0]]

        cv_results = model_selection.cross_val_score(model, X_train_, Y_train, cv=kfold, scoring='roc_auc')
        
        sheet.write(r,c,cv_results.mean())
        
        msg = "%s %s: %f (%f)\n" % (kind, name, cv_results.mean(), cv_results.std())
        print(msg)
        output.write(msg)
        model.fit(X_train_, Y_train)
        joblib.dump(model,'./handpkl/Endo'+name+kind+'10.pkl')
        
        Y_pred = model.predict(X_train_)
        #prob = model.predict_prob(X_train_)
        print("Accuracy: " + repr(accuracy_score(Y_train, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_train, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_train, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_train, Y_pred)))
        print("AUC: " + repr(roc_auc_score(Y_train, Y_pred)))
        print("Sensitivity: " + repr(recall_score(Y_train, Y_pred)))
        tn, fp, fn, tp = confusion_matrix(Y_train, Y_pred).ravel()
        print("Specificity: " + repr(tn / (tn + fp)))
        
        sheet_train.write(r,c,roc_auc_score(Y_train, Y_pred))
        
        Y_pred = model.predict(X_validate_)    
        #prob = model.predict_prob(X_validate_)
        print("Accuracy: " + repr(accuracy_score(Y_validate, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_validate, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_validate, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_validate, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_validate,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_validate,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_validate,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_validate.write(r,c,roc_auc_score(Y_validate,Y_pred))
        
        Y_pred = model.predict(X_test_)
        #prob = model.predict_prob(X_test_)
        print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_test,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_test,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_test.write(r,c,roc_auc_score(Y_test,Y_pred))
        
        r = r + 1
    c = c + 1
    r = 0

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

MIM GLM: 0.783917 (0.026441)

Accuracy: 0.7257575757575757
Average Precision Score: 0.6205517447153135
Kappa: 0.44086645573175065
Hamming Loss: 0.27424242424242423
AUC: 0.7185550935550935
Sensitivity: 0.6486486486486487
Specificity: 0.7884615384615384
Accuracy: 0.7349397590361446
Average Precision Score: 0.6353760908294834
Kappa: 0.4704176334106728
Hamming Loss: 0.26506024096385544
AUC0.7371345029239766
Sensitivity0.7631578947368421
Specificity0.7111111111111111
Accuracy: 0.7590361445783133
Average Precision Score: 0.6602953166047649
Kappa: 0.5185614849187935
Hamming Loss: 0.24096385542168675
AUC0.7614035087719299
Sensitivity0.7894736842105263
Specificity0.7333333333333333
MIFS GLM: 0.764341 (0.027688)

Accuracy: 0.6318181818181818
Average Precision Score: 0.5283190533190534
Kappa: 0.21117865785280054
Hamming Loss: 0.36818181818181817
AUC: 0.5996250371250371
Sensitivity: 0.28716216216216217
Specificity: 0.9120879120879121
Accuracy: 0.6024096385542169
Average Precision Score: 0.51676882

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

DISR GLM: 0.787197 (0.029017)

Accuracy: 0.7151515151515152
Average Precision Score: 0.611750190629501
Kappa: 0.4123219157320399
Hamming Loss: 0.28484848484848485
AUC: 0.701997326997327
Sensitivity: 0.5743243243243243
Specificity: 0.8296703296703297
Accuracy: 0.7108433734939759
Average Precision Score: 0.6157557863031071
Kappa: 0.4103019538188277
Hamming Loss: 0.2891566265060241
AUC0.7026315789473684
Sensitivity0.6052631578947368
Specificity0.8
Accuracy: 0.7228915662650602
Average Precision Score: 0.6265513180541716
Kappa: 0.4383642247719918
Hamming Loss: 0.27710843373493976
AUC0.7178362573099415
Sensitivity0.6578947368421053
Specificity0.7777777777777778
MIM LDA: 0.775109 (0.022798)

Accuracy: 0.7181818181818181
Average Precision Score: 0.6113704379971503
Kappa: 0.4295963125418184
Hamming Loss: 0.2818181818181818
AUC: 0.714527027027027
Sensitivity: 0.6790540540540541
Specificity: 0.75
Accuracy: 0.7590361445783133
Average Precision Score: 0.6590981149478297
Kappa: 0.5205083766608898
Ha

CMIM DT: 0.582592 (0.047437)

Accuracy: 1.0
Average Precision Score: 1.0
Kappa: 1.0
Hamming Loss: 0.0
AUC: 1.0
Sensitivity: 1.0
Specificity: 1.0
Accuracy: 0.5903614457831325
Average Precision Score: 0.5131723064506831
Kappa: 0.18486424032351245
Hamming Loss: 0.40963855421686746
AUC0.5935672514619883
Sensitivity0.631578947368421
Specificity0.5555555555555556
Accuracy: 0.5662650602409639
Average Precision Score: 0.49495085605580214
Kappa: 0.12987769365171808
Hamming Loss: 0.43373493975903615
AUC0.5652046783625732
Sensitivity0.5526315789473685
Specificity0.5777777777777777
ICAP DT: 0.597006 (0.039249)

Accuracy: 1.0
Average Precision Score: 1.0
Kappa: 1.0
Hamming Loss: 0.0
AUC: 1.0
Sensitivity: 1.0
Specificity: 1.0
Accuracy: 0.6144578313253012
Average Precision Score: 0.5304303337752926
Kappa: 0.23281340265742356
Hamming Loss: 0.3855421686746988
AUC0.6178362573099415
Sensitivity0.6578947368421053
Specificity0.5777777777777777
Accuracy: 0.5662650602409639
Average Precision Score: 0.4949508

...

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the op

DISR NNet: 0.798984 (0.030523)



/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.7318181818181818
Average Precision Score: 0.6288391886997465
Kappa: 0.4500724951512983
Hamming Loss: 0.2681818181818182
AUC: 0.7218406593406593
Sensitivity: 0.625
Specificity: 0.8186813186813187
Accuracy: 0.7108433734939759
Average Precision Score: 0.6127223575743417
Kappa: 0.4175438596491228
Hamming Loss: 0.2891566265060241
AUC0.7087719298245614
Sensitivity0.6842105263157895
Specificity0.7333333333333333
Accuracy: 0.7469879518072289
Average Precision Score: 0.6510222968688409
Kappa: 0.48930559624963377
Hamming Loss: 0.25301204819277107
AUC0.7441520467836258
Sensitivity0.7105263157894737
Specificity0.7777777777777778
MIM RF: 0.726209 (0.020876)

Accuracy: 0.9818181818181818
Average Precision Score: 0.9746578965328966
Kappa: 0.9631531933899062
Hamming Loss: 0.01818181818181818
AUC: 0.9803608553608554
Sensitivity: 0.9662162162162162
Specificity: 0.9945054945054945
Accuracy: 0.6506024096385542
Average Precision Score: 0.5564939182567591
Kappa: 0.2889217134416544
Hamming Loss: 

JMI BST: 0.737696 (0.040607)

Accuracy: 0.7803030303030303
Average Precision Score: 0.6815293762662185
Kappa: 0.55434478904722
Hamming Loss: 0.2196969696969697
AUC: 0.7762102762102762
Sensitivity: 0.7364864864864865
Specificity: 0.8159340659340659
Accuracy: 0.7108433734939759
Average Precision Score: 0.6121353836398225
Kappa: 0.4199184624344787
Hamming Loss: 0.2891566265060241
AUC0.7108187134502923
Sensitivity0.7105263157894737
Specificity0.7111111111111111
Accuracy: 0.7831325301204819
Average Precision Score: 0.6884908053265694
Kappa: 0.564938846825859
Hamming Loss: 0.21686746987951808
AUC0.7836257309941521
Sensitivity0.7894736842105263
Specificity0.7777777777777778
CMIM BST: 0.739368 (0.050669)

Accuracy: 0.7803030303030303
Average Precision Score: 0.6801090381598855
Kappa: 0.5557515551016619
Hamming Loss: 0.2196969696969697
AUC: 0.7777880902880904
Sensitivity: 0.7533783783783784
Specificity: 0.8021978021978022
Accuracy: 0.6746987951807228
Average Precision Score: 0.5787359966180511


In [14]:
book.save(file)
output.close()

In [19]:
print(idx)

[69, 73, 4668, 4732, 4679, 4743, 51, 54, 57, 4716]
